In [7]:
# continuing the task here

### Import libraries

In [1]:
# Import libraries 
import pandas as pd
import numpy as np
import os

### Import orders_products_merged3.pkl

In [2]:
path = r'C:\Users\myra_\Documents\Instacart Basket Analysis 02-2023'

In [4]:
# Import the dataset 'orders_products_merged3.pkl'
orders_products_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared data', 'orders_products_merged3.pkl'))

In [5]:
orders_products_merged = orders_products_merged[orders_products_merged.loyalty_flag.notnull()]

In [6]:
orders_products_merged['loyalty_flag'].value_counts(dropna = False)

Regular customer    15891507
Loyal customer      10294027
New customer         6249525
Name: loyalty_flag, dtype: int64

The 11 rows that had null values are now deleted from the dataframe

## Finding out the aggregated mean of the “order_number” column grouped by “department_id'' for the whole dataframe

In [8]:
orders_products_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


For the subset (working done in script 4.8), there were eight department ids. This means that for the first million entries, there were only eight departments and their data. However, in the overall dataframe the data consisted of 21 departments. Moreover, the means of first million rows versus the entire dataframe vary only slightly. 

In [11]:
orders_products_merged.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,busiest_day,busiest_days_of_week,busiest_period_of_day,max_order,loyalty_flag,loyalty flag
0,40.0,382.0,29.0,1.0,15.0,23.0,10070,1.0,1.0,Organic 1% Low Fat Milk,84.0,16.0,5.9,both,Regularly busy,Busiest days,Most orders,29.0,Regular customer,NaN
1,235.0,58701.0,20.0,5.0,9.0,9.0,10070,7.0,1.0,Organic 1% Low Fat Milk,84.0,16.0,5.9,both,Regularly busy,Regularly busy days,Fewest orders,21.0,Regular customer,NaN
2,581.0,76141.0,3.0,6.0,17.0,7.0,10070,3.0,0.0,Organic 1% Low Fat Milk,84.0,16.0,5.9,both,Regularly busy,Regularly busy days,Average orders,27.0,Regular customer,NaN
3,649.0,135836.0,1.0,4.0,7.0,NaN,10070,1.0,0.0,Organic 1% Low Fat Milk,84.0,16.0,5.9,both,Least busy,Least busy days,Average orders,17.0,Regular customer,NaN
4,1662.0,2565.0,7.0,5.0,14.0,13.0,10070,10.0,1.0,Organic 1% Low Fat Milk,84.0,16.0,5.9,both,Regularly busy,Regularly busy days,Most orders,11.0,Regular customer,NaN


In [12]:
# Deleting column name 'loyalty flag' since that was created accidentally
del orders_products_merged['loyalty flag']

In [15]:
# Checking the basic statistics of the product prices for each loyalty category 
orders_products_merged.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

The new customers on average go for products with higher prices and the loyal customers buy products that are cheaper. However, the maximum price of a product is 99999 which has skewed the distribution considerably. 

In [16]:
orders_products_merged['avg_order'] = orders_products_merged.groupby(['user_id'])['prices'].transform('mean')

In [47]:
orders_products_merged[['user_id', 'avg_order']].head()

MemoryError: Unable to allocate 3.38 GiB for an array with shape (14, 32435059) and data type float64

## Creating a spending flag for each user based on the average price across all their orders

In [22]:
orders_products_merged.loc[orders_products_merged['avg_order'] < 10, 'spending_flag'] = 'Low spender'

In [25]:
orders_products_merged.loc[orders_products_merged['avg_order'] >= 10, 'spending_flag'] = 'High spender'

In [27]:
orders_products_merged['spending_flag'].value_counts(dropna = False)

Low spender     31800381
High spender      634678
Name: spending_flag, dtype: int64

There are 634678 people that are high spenders and the rest are marked as low spenders.

## Creating an order frequency flag that marks the regularity of a user’s ordering behavior

In [30]:
orders_products_merged['median_days_order'] = orders_products_merged.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [32]:
orders_products_merged.loc[orders_products_merged['median_days_order'] > 20, 'Order_freq_flag'] = 'Non-frequent customer'

In [33]:
orders_products_merged.loc[(orders_products_merged['median_days_order'] > 10) & (orders_products_merged['median_days_order'] <= 20) , 'Order_freq_flag'] = 'Regular customer'

In [34]:
orders_products_merged.loc[orders_products_merged['median_days_order'] <= 10, 'Order_freq_flag'] = 'Frequent customer'

In [36]:
orders_products_merged['Order_freq_flag'].value_counts(dropna = False)

Frequent customer        21578273
Regular customer          7217066
Non-frequent customer     3639720
Name: Order_freq_flag, dtype: int64

There are 21578273 customers who are frequent customers and order again under less than 10 days.

In [46]:
orders_products_merged[['user_id', 'median_days_order', 'Order_freq_flag']].head()

MemoryError: Unable to allocate 3.38 GiB for an array with shape (14, 32435059) and data type float64

## Saving the dataset in pkl

In [48]:
# Export data to pkl
orders_products_merged.to_pickle(os.path.join(path, '02 Data','Prepared data', 'orders_products_merged4.pkl'))